<a href="https://colab.research.google.com/github/sachinsaraff10/hotelbookings/blob/master/hotelbookings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
jessemostipak_hotel_booking_demand_path = kagglehub.dataset_download('jessemostipak/hotel-booking-demand')

print('Data source import complete.')


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
df2=pd.read_csv('/kaggle/input/hotel-booking-demand/hotel_bookings.csv')

Step 1: Checking for missing values and establishing which columns may need dropping based on NaN count


In [ ]:
nan_counts = df2.isna().sum()

# Sort columns by the number of NaN values in descending order
nan_counts_sorted = nan_counts.sort_values(ascending=False)
df2.drop(columns=['company','agent'],inplace=True)
df2.dropna(inplace=True)

In [ ]:
df2.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
days_in_waiting_list              

I drop irrelevent categorical and numerical variables

In [ ]:
cat_columns = df2.select_dtypes(include=['object']).columns
cat_columns=cat_columns.drop(['hotel','country','meal','reservation_status_date','arrival_date_month','market_segment','distribution_channel'])
df2.drop(columns=['lead_time','arrival_date_year','arrival_date_week_number','arrival_date_month',
                 'stays_in_weekend_nights','stays_in_week_nights','adr'],inplace=True)

One-hot encode categorical variables

In [ ]:
X = pd.get_dummies(df2,columns=cat_columns,drop_first=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,recall_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV


In [ ]:
X.drop(columns=['is_canceled'],inplace=True)
y=df2.is_canceled

In [ ]:
X.dtypes

hotel                              object
arrival_date_day_of_month           int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
booking_changes                     int64
days_in_waiting_list                int64
required_car_parking_spaces         int64
total_of_special_requests           int64
reservation_status_date            object
reserved_room_type_B                 bool
reserved_room_type_C                 bool
reserved_room_type_D                 bool
reserved_room_type_E                 bool
reserved_room_type_F                 bool
reserved_room_type_G                 bool
reserved_room_type_H              

Splitting data into train and test sets

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Scaling numerical variables to use in logistic regression

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include='number'))
X_test_scaled = scaler.fit_transform(X_test.select_dtypes(include='number'))
X_train_scaled = pd.DataFrame(X_train_scaled,columns=X_train.select_dtypes(include='number').columns,index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled,columns=X_test.select_dtypes(include='number').columns,index=X_test.index)

Grouping together one-hot encoded variables

In [ ]:
bool_cols=X_train.select_dtypes(include='bool').columns

In [ ]:
final_train_df = pd.concat([X_train_scaled,X_train[bool_cols]],axis=1)
final_test_df = pd.concat([X_test_scaled,X_test[bool_cols]],axis=1)

In [ ]:
X_train[bool_cols]

,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_B,...,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,reservation_status_Check-Out,reservation_status_No-Show
2499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
36906,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
16363,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
25818,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
33552,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77310,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
110760,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
104186,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
861,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [ ]:
final_train_df.dtypes

arrival_date_day_of_month         float64
adults                            float64
children                          float64
babies                            float64
is_repeated_guest                 float64
previous_cancellations            float64
previous_bookings_not_canceled    float64
booking_changes                   float64
days_in_waiting_list              float64
required_car_parking_spaces       float64
total_of_special_requests         float64
reserved_room_type_B                 bool
reserved_room_type_C                 bool
reserved_room_type_D                 bool
reserved_room_type_E                 bool
reserved_room_type_F                 bool
reserved_room_type_G                 bool
reserved_room_type_H                 bool
reserved_room_type_L                 bool
reserved_room_type_P                 bool
assigned_room_type_B                 bool
assigned_room_type_C                 bool
assigned_room_type_D                 bool
assigned_room_type_E              

In [ ]:
from sklearn.linear_model import LogisticRegression
# Initialize the multinomial logistic regression model
model = LogisticRegression(max_iter=100)

# Train the model
model.fit(final_train_df, y_train)

# Make predictions
predictions = model.predict(final_test_df)

print("Predictions:", predictions)

Predictions: [1 1 0 ... 0 1 0]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

Final accuracy results

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [ ]:
from sklearn.metrics import log_loss

Computing log-loss and cross-validating the data with other popular classification algorithms

In [ ]:
y_pred_proba = model.predict_proba(final_test_df)[:,1]
score_ll = log_loss(y_test, y_pred_proba)

In [ ]:
score_ll

0.0003268286238863165

In [ ]:
X = X.select_dtypes(exclude=['object'])

In [ ]:
models = [('KNN', KNeighborsClassifier()),
          ('Decisiontree', DecisionTreeClassifier(random_state=123456)),
          ('RF', RandomForestClassifier(random_state=123456)),
          ('GB', GradientBoostingClassifier(random_state = 12345)),
          ("XGBoost", XGBClassifier(random_state=123456))]
results = []
names = []

def crossvalidator(models,x):
    name, model =models[x]
    print(models[x])
    kfold = KFold(n_splits=7)
    cv_results = cross_val_score(model, X, y, cv=kfold)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


In [ ]:
for _ in range(len(models)):
    crossvalidator(models,len(results))

('KNN', KNeighborsClassifier())
KNN: 0.955425 (0.051509)
('Decisiontree', DecisionTreeClassifier(random_state=123456))
Decisiontree: 1.000000 (0.000000)
('RF', RandomForestClassifier(random_state=123456))
RF: 1.000000 (0.000000)
('GB', GradientBoostingClassifier(random_state=12345))
GB: 1.000000 (0.000000)
('XGBoost', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=Non

In [ ]:
xgboost_mod = XGBClassifier(random_state=12345)
xgboost_mod.fit(final_train_df, y_train)
y_pred = xgboost_mod.predict(final_test_df)
conf_mat = confusion_matrix(y_pred,y_test)
print("True Positive : ", conf_mat[1, 1])
print("True Negative : ", conf_mat[0, 0])
print("False Positive: ", conf_mat[0, 1])
print("False Negative: ", conf_mat[1, 0])

True Positive :  8929
True Negative :  14851
False Positive:  0
False Negative:  0


In [ ]:
print(classification_report(xgboost_mod.predict(final_test_df),y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14851
           1       1.00      1.00      1.00      8929

    accuracy                           1.00     23780
   macro avg       1.00      1.00      1.00     23780
weighted avg       1.00      1.00      1.00     23780



In [ ]:
KNN_mod = KNeighborsClassifier()
KNN_mod.fit(final_train_df, y_train)
y_pred = KNN_mod.predict(final_test_df)
conf_mat = confusion_matrix(y_pred,y_test)
print("True Positive : ", conf_mat[1, 1])
print("True Negative : ", conf_mat[0, 0])
print("False Positive: ", conf_mat[0, 1])
print("False Negative: ", conf_mat[1, 0])

True Positive :  8751
True Negative :  14836
False Positive:  178
False Negative:  15


In [ ]:
print(classification_report(KNN_mod.predict(final_test_df),y_test))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15014
           1       0.98      1.00      0.99      8766

    accuracy                           0.99     23780
   macro avg       0.99      0.99      0.99     23780
weighted avg       0.99      0.99      0.99     23780



In [ ]:
RF_mod = RandomForestClassifier(random_state=12345)
RF_mod.fit(final_train_df, y_train)
y_pred = RF_mod.predict(final_test_df)
conf_mat = confusion_matrix(y_pred,y_test)
print("True Positive : ", conf_mat[1, 1])
print("True Negative : ", conf_mat[0, 0])
print("False Positive: ", conf_mat[0, 1])
print("False Negative: ", conf_mat[1, 0])

True Positive :  8929
True Negative :  14851
False Positive:  0
False Negative:  0


In [ ]:
print(classification_report(RF_mod.predict(final_test_df),y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14851
           1       1.00      1.00      1.00      8929

    accuracy                           1.00     23780
   macro avg       1.00      1.00      1.00     23780
weighted avg       1.00      1.00      1.00     23780



In [ ]:
DT_mod = DecisionTreeClassifier(random_state=12345)
DT_mod.fit(final_train_df, y_train)
y_pred = d_mod.predict(final_test_df)
conf_mat = confusion_matrix(y_pred,y_test)
print("True Positive : ", conf_mat[1, 1])
print("True Negative : ", conf_mat[0, 0])
print("False Positive: ", conf_mat[0, 1])
print("False Negative: ", conf_mat[1, 0])